<a href="https://colab.research.google.com/github/RaquelRufino/InformationRetrieval/blob/master/ri_lab_05/lab05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
import pandas as pd
import numpy as np
import nltk
import re
import collections
import heapq
import time
import bisect
import math
import random
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
result = pd.read_csv("https://raw.githubusercontent.com/RaquelRufino/InformationRetrieval/master/ri_lab_04/results.csv")
tknz = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
ranking = {}
n = 0

for text in result.text:
  words = [word for word in tknz.tokenize(text.lower())
           if not bool(re.search(r'\d', word))
           and word not in stopwords and len(word) >= 3]  
  n += 1
  for t in words:
    if t not in ranking.keys():
      ranking[t] = []
    ranking[t].append(n)
    
for elem in ranking.items():
  d = dict(collections.Counter(elem[1]))
  ranking[elem[0]] = list(d.items())
 

In [0]:
queries = ["política", "presidente", "educação", "ministério", "empresa"]

### Calcule as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com as 4 métricas que vimos na aula. Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204). Qual métrica você acha que obteve os melhores resultados? Por que? (20 pts.)


In [0]:

def n_docs_word(word, ranking):
  return len(ranking[word])

def n_docs_both(a, b, ranking):
  docs_a = [doc for doc,score in ranking[a]]
  docs_b = [doc for doc,score in ranking[b]]
  n = 0
  
  for doc in docs_a:
    if doc in docs_b:
      n += 1
      
  return n

In [0]:
def calc_mim(a, b, ranking):
  n_a = n_docs_word(a, ranking)
  n_b = n_docs_word(b, ranking)
  n_ab = n_docs_both(a, b, ranking)
  quo = (n_a * n_b)
  mim = n_ab / quo if quo != 0 else 0
  return (mim,b)
  
def calc_emim(a, b, ranking, n):
  n_a = n_docs_word(a, ranking)
  n_b = n_docs_word(b, ranking)
  n_ab = n_docs_both(a, b, ranking)
  quo = (n_a * n_b)
  exp = n * (n_ab / quo) if quo != 0 else 0
  emim = n_ab * np.log10(exp) if exp != 0 else 0
  return (emim, b)
  
def calc_x(a, b, ranking, n):
  n_a = n_docs_word(a, ranking)
  n_b = n_docs_word(b, ranking)
  n_ab = n_docs_both(a, b, ranking)
  quo = (n_a * n_b)
  x = (n_ab - (1 / n) * n_a * n_b) ** 2 / quo if quo != 0 else 0
  return (x,b)
  
def calc_dice(a, b, ranking):
  n_a = n_docs_word(a, ranking)
  n_b = n_docs_word(b, ranking)
  n_ab = n_docs_both(a, b, ranking)
  dice = n_ab / (n_a + n_b)
  return (dice,b)

In [0]:
def create_metrics_table(query, ranking, n):
  top_10_table = pd.DataFrame()
  MIM = []
  EMIM = []
  X = []
  Dice = []
  
  for word in ranking:
    if word != query:
      bisect.insort(MIM, calc_mim(query, word, ranking))
      bisect.insort(EMIM,calc_emim(query, word, ranking, n))
      bisect.insort(X,calc_x(query, word, ranking, n))
      bisect.insort(Dice,calc_dice(query, word, ranking))

  MIM.reverse()
  EMIM.reverse()
  X.reverse()
  Dice.reverse()
  
  top_10_table['MIM'] = [word[1] for word in MIM[:10]]
  top_10_table['EMIM'] = [word[1] for word in EMIM[:10]]
  top_10_table['X²'] = [word[1] for word in X[:10]]
  top_10_table['Dice'] = [word[1] for word in Dice[:10]]
  
  top_10_table.index += 1
  
  return top_10_table

In [0]:
n = result.text.count()
table_1 = create_metrics_table(queries[0], ranking, n)
table_2 = create_metrics_table(queries[1], ranking, n)
table_3 = create_metrics_table(queries[2], ranking, n)
table_4 = create_metrics_table(queries[3], ranking, n)
table_5 = create_metrics_table(queries[4], ranking, n)

1.1 Top-10 palavras associadas a 'política'


In [136]:
table_1

,MIM,EMIM,X²,Dice
1,útero,político,político,país
2,úteis,cada,desigualdade,outros
3,úlceras,país,cientista,vez
4,ótimas,bolsonaro,eleições,cada
5,ópio,nacional,problemas,governo
6,óbvios,governo,força,menos
7,óbitos,estado,eleição,pode
8,ítalo,outros,estar,desde
9,índios,estar,economia,parte
10,êxito,economia,democracia,ser


1.1 Top-10 palavras associadas a 'presidente'


In [137]:
table_2

,MIM,EMIM,X²,Dice
1,ótimas,bolsonaro,bolsonaro,feira
2,ódios,ministro,ministro,governo
3,óbvios,governo,deputado,bolsonaro
4,óbvias,feira,presidência,ainda
5,óbvia,político,governo,sobre
6,óbitos,segurança,político,país
7,íntegra,afirmou,câmara,ser
8,índole,deputado,vice,ministro
9,ídolos,câmara,segurança,ter
10,ícone,presidência,secretário,outros


1.1 Top-10 palavras associadas a  'educação''


In [138]:
table_3

,MIM,EMIM,X²,Dice
1,óbvios,alunos,alunos,alunos
2,óbvias,escolas,vélez,escolas
3,óbvia,social,escolas,social
4,étnicos,escola,mec,mudanças
5,étnico,discurso,aulas,escola
6,átomo,governo,aula,discurso
7,árbitros,mudanças,aluno,comissão
8,zombava,comissão,pasta,necessidade
9,zeina,vélez,estudantes,jovens
10,zeca,pasta,educacional,pasta


1.1 Top-10 palavras associadas a 'ministério'


In [139]:
table_4

,MIM,EMIM,X²,Dice
1,úteis,federal,pasta,federal
2,óbvios,ministro,federal,ministro
3,íntegra,público,feita,público
4,ídolos,militares,valores,militares
5,étnico,valores,vieira,valores
6,ético,feita,promovido,governo
7,ânus,psl,lamenta,disse
8,ângulos,governo,psl,rio
9,ângelo,pasta,cidadania,justiça
10,áudios,disse,ministro,chefe


1.1 Top-10 palavras associadas a  'empresa'


In [140]:
table_5

,MIM,EMIM,X²,Dice
1,ósseas,reais,reais,reais
2,óbitos,empresas,bilhão,empresas
3,ídolos,milhões,acionistas,milhões
4,épocas,bilhões,bilhões,bilhões
5,átomo,dólares,empresas,valor
6,áthila,valor,dólares,dólares
7,árbitros,cerca,fabricação,cerca
8,ágio,mercado,fabricante,mercado
9,ábaton,bilhão,tornando,além
10,zonzo,acionistas,obteve,seis


Em geral, o Dice parece ser a melhor escolha, pois seus resultados estão mais próximos em termos de semântica aos termos originais (os resultados são mais coerentes)

### De acordo com a métrica que deu os melhores resultados na sua opinião, execute agora cada consulta (usando a abordagem documento- ou termo-por-vez) expandido-a com: os top-3, top-5 e top-10 documentos. O que acontece com a precisão dos resultados em cada caso? Aumenta ou diminui? Justifique bem sua resposta.

Algortimo escolhido: Termo-por-vez

In [0]:
def term_a_time(query, indexes, k):    
    a = {} 
    query_indexes = [] 
    rank = []
    
    for word in query.split(" "):
      if word in indexes.keys():
          query_indexes.append(indexes[word])
    
    for index in query_indexes:
      for i in index:
          doc = i[0]
          if doc not in a.keys():
              a[doc] = 0
          a[doc] += i[1]
    
    for (doc, score) in a.items():
      heapq.heappush(rank, (score, doc))
        
    return heapq.nlargest(k, rank)

In [0]:
def top_docs(query, k):
  
  top_docs = [doc for score,doc in term_a_time(query, ranking, k)]
  top_docs_ranking = {} 
  
  for word in ranking:
    top_docs_ranking[word] = []
    for doc_score in ranking[word]:
      if doc_score[0] in top_docs:
        top_docs_ranking[word].append(doc_score)
        
  return top_docs_ranking

Top-3 Documentos

In [0]:
k = 3
ranking_docs1 = top_docs(queries[0], k)
ranking_docs2 = top_docs(queries[1], k)
ranking_docs3 = top_docs(queries[2], k)
ranking_docs4 = top_docs(queries[3], k)
ranking_docs5 = top_docs(queries[4], k)

In [0]:
table_top1 = create_metrics_table(queries[0], ranking_docs1, k)
table_top2 = create_metrics_table(queries[1], ranking_docs2, k)
table_top3 = create_metrics_table(queries[2], ranking_docs3, k)
table_top4 = create_metrics_table(queries[3], ranking_docs4, k)
table_top5 = create_metrics_table(queries[4], ranking_docs5, k)

Top-10 palavras associadas aos Top-3 Documentos de 'política'

In [145]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_1['Dice']
table_dice['Dice after Term A Time'] = table_top1['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,país,vez
2,outros,vamos
3,vez,tudo
4,cada,tornou
5,governo,todos
6,menos,ter
7,pode,sobre
8,desde,ser
9,parte,segurança
10,ser,quer


Top-10 palavras associadas aos Top-3 Documentos de 'presidente'

In [146]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_2['Dice']
table_dice['Dice after Term A Time'] = table_top2['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,feira,vêm
2,governo,vão
3,bolsonaro,vive
4,ainda,violência
5,sobre,vida
6,país,vezes
7,ser,vez
8,ministro,verdade
9,ter,tudo
10,outros,três


Top-10 palavras associadas aos Top-3 Documentos de 'educação''


In [147]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_3['Dice']
table_dice['Dice after Term A Time'] = table_top3['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,alunos,todo
2,escolas,social
3,social,ser
4,mudanças,qualidade
5,escola,proposta
6,discurso,projeto
7,comissão,parece
8,necessidade,onde
9,jovens,fazendo
10,pasta,faz


Top-10 palavras associadas aos Top-3 Documentos de 'ministério'

In [148]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_4['Dice']
table_dice['Dice after Term A Time'] = table_top4['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,federal,vez
2,ministro,ter
3,público,ser
4,militares,reais
5,valores,pública
6,governo,projeto
7,disse,primeiro
8,rio,política
9,justiça,parte
10,chefe,ministro


Top-10 palavras associadas aos Top-3 Documentos de 'empresa'

In [149]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_5['Dice']
table_dice['Dice after Term A Time'] = table_top5['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,reais,sobre
2,empresas,reais
3,milhões,nesta
4,bilhões,maior
5,valor,feira
6,dólares,dólares
7,cerca,dois
8,mercado,desde
9,além,custo
10,seis,cerca


Top-5 Documentos

In [0]:
k = 5
ranking_docs1 = top_docs(queries[0], k)
ranking_docs2 = top_docs(queries[1], k)
ranking_docs3 = top_docs(queries[2], k)
ranking_docs4 = top_docs(queries[3], k)
ranking_docs5 = top_docs(queries[4], k)

In [0]:

table_top1 = create_metrics_table(queries[0], ranking_docs1, k)
table_top2 = create_metrics_table(queries[1], ranking_docs2, k)
table_top3 = create_metrics_table(queries[2], ranking_docs3, k)
table_top4 = create_metrics_table(queries[3], ranking_docs4, k)
table_top5 = create_metrics_table(queries[4], ranking_docs5, k)

Top-10 palavras associadas aos Top-5 Documentos de 'política'

In [152]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_1['Dice']
table_dice['Dice after Term A Time'] = table_top1['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,país,todos
2,outros,sobre
3,vez,ser
4,cada,quatro
5,governo,presidente
6,menos,país
7,pode,outros
8,desde,outro
9,parte,oposição
10,ser,ministro


Top-10 palavras associadas aos Top-5 Documentos de 'presidente'

In [153]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_2['Dice']
table_dice['Dice after Term A Time'] = table_top2['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,feira,vão
2,governo,vezes
3,bolsonaro,verdade
4,ainda,ter
5,sobre,sobre
6,país,sido
7,ser,ser
8,ministro,quase
9,ter,pode
10,outros,pessoas


Top-10 palavras associadas aos Top-5 Documentos de 'educação'

In [154]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_3['Dice']
table_dice['Dice after Term A Time'] = table_top3['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,alunos,ser
2,escolas,qualidade
3,social,onde
4,mudanças,todo
5,escola,ter
6,discurso,social
7,comissão,sobre
8,necessidade,segundo
9,jovens,pessoas
10,pasta,parece


Top-10 palavras associadas aos Top-5 Documentos de 'ministério'

In [155]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_4['Dice']
table_dice['Dice after Term A Time'] = table_top4['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,federal,desde
2,ministro,ter
3,público,social
4,militares,ser
5,valores,semana
6,governo,segundo
7,disse,país
8,rio,pasta
9,justiça,parte
10,chefe,médio


Top-10 palavras associadas aos Top-5 Documentos de 'empresa'

In [156]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_5['Dice']
table_dice['Dice after Term A Time'] = table_top5['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,reais,nesta
2,empresas,dólares
3,milhões,ações
4,bilhões,últimos
5,valor,valor
6,dólares,total
7,cerca,sobre
8,mercado,segurança
9,além,segundo
10,seis,reais


Top-10 Documentos

In [0]:
k = 10
ranking_docs1 = top_docs(queries[0], k)
ranking_docs2 = top_docs(queries[1], k)
ranking_docs3 = top_docs(queries[2], k)
ranking_docs4 = top_docs(queries[3], k)
ranking_docs5 = top_docs(queries[4], k)

In [0]:

table_top1 = create_metrics_table(queries[0], ranking_docs1, k)
table_top2 = create_metrics_table(queries[1], ranking_docs2, k)
table_top3 = create_metrics_table(queries[2], ranking_docs3, k)
table_top4 = create_metrics_table(queries[3], ranking_docs4, k)
table_top5 = create_metrics_table(queries[4], ranking_docs5, k)

Top-10 palavras associadas aos Top-10 Documentos de 'política'

In [159]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_1['Dice']
table_dice['Dice after Term A Time'] = table_top1['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,país,país
2,outros,sobre
3,vez,ser
4,cada,quatro
5,governo,pessoas
6,menos,menos
7,pode,maior
8,desde,ainda
9,parte,agora
10,ser,todos


Top-10 palavras associadas aos Top-10 Documentos de 'presidente'

In [160]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_2['Dice']
table_dice['Dice after Term A Time'] = table_top2['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,feira,vezes
2,governo,sobre
3,bolsonaro,ser
4,ainda,contra
5,sobre,anos
6,país,ter
7,ser,quase
8,ministro,governo
9,ter,fazer
10,outros,desde


Top-10 palavras associadas aos Top-10 Documentos de 'educação'

In [161]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_3['Dice']
table_dice['Dice after Term A Time'] = table_top3['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,alunos,sobre
2,escolas,ser
3,social,presidente
4,mudanças,onde
5,escola,ainda
6,discurso,ter
7,comissão,segundo
8,necessidade,pessoas
9,jovens,outras
10,pasta,maior


Top-10 palavras associadas aos Top-10 Documentos de 'ministério'

In [162]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_4['Dice']
table_dice['Dice after Term A Time'] = table_top4['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,federal,ser
2,ministro,governo
3,público,ter
4,militares,país
5,valores,parte
6,governo,federal
7,disse,anos
8,rio,ainda
9,justiça,três
10,chefe,social


Top-10 palavras associadas aos Top-10 Documentos de 'empresa'

In [163]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_5['Dice']
table_dice['Dice after Term A Time'] = table_top5['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,reais,reais
2,empresas,nesta
3,milhões,dólares
4,bilhões,bilhões
5,valor,sobre
6,dólares,segundo
7,cerca,maior
8,mercado,feira
9,além,além
10,seis,passado


Para o top-3  a qualidade dos resultados não é o muito do que era esperado.
Para os top-5 e top-10 a qualidade dos resultados mostram mais satifastórios podendo ser justificado pela consideração de documentos que realmente descrevem o contexto da palavra consultada.